In [9]:
from bs4 import BeautifulSoup
from collections import defaultdict
import json
import re
import requests
from time import sleep
import datetime as dt
import copy
from pymongo import MongoClient
import pickle

In [138]:
beer_dict = defaultdict(set)
url = 'https://www.beeradvocate.com/beer/'
resp = requests.get(url)
soup = BeautifulSoup(resp.text,"html")
next_exists = True

/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/ubuntu/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [139]:
while True:
    try:
        url = 'https://www.beeradvocate.com' + soup.find(text=re.compile('next')).parent['href']
    except:
        break   # this should only happen if next link is no longer clickable
    
    soup = soup.find('div', {'id': 'rating_fullview'})

    for comment in soup.findAll('div',{'class': 'user-comment'}):
        beer_url = comment.find('h6').a['href']
        beer_name = comment.find('h6').text
        beer_dict[beer_name].add(beer_url)
    sleep(.1)

    if next_exists==True:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,"html")

/home/ubuntu/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/ubuntu/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


https://www.beeradvocate.com/beer/?start=450


AttributeError: 'NoneType' object has no attribute 'findAll'

In [205]:
for k in beer_dict:
    beer_dict[k] = list(beer_dict[k])

In [142]:
beer_dict = defaultdict(set)

In [109]:
url = 'https://www.beeradvocate.com/lists/top/'
resp = requests.get(url)

soup = BeautifulSoup(resp.text,'lxml')
soup = soup.find('div',{'class': 'secondaryContent'})

vals = []
for op in soup.find('form').findAll('option'):   # get beer style number for top style lists
    if op['value'] != '':
        vals.append(op['value'])

In [114]:
top_style_urls = [] # create urls using style numbers

for val in vals:
    top_style_urls.append('https://www.beeradvocate.com/lists/style/{}/'.format(val))

top_style_urls.append('https://www.beeradvocate.com/lists/fame/') # add two more top lists to get beer urls from
top_style_urls.append('https://www.beeradvocate.com/lists/top/')

In [187]:
beer_dict = defaultdict(set)

In [191]:
for url in top_style_urls:
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text,'lxml')
    table = soup.find('table')

    for tr in table.findAll('tr')[2:]:
        link = tr.findAll('td')[1].find('a')
        beer_dict[link.text].add(link['href'])
        
    sleep(.1)

In [206]:
beer_dict["Delirium Tremens"]

['/beer/profile/180/1385/']

In [545]:
unique_beer_dict = {}

for k in beer_dict:
    for l in beer_dict[k]:
        unique_beer_dict[l] = {}
        unique_beer_dict[l]['beer_name'] = k

In [20]:
unique_beer_dict

{'/beer/profile/12011/25309/': {'abv': '4.00%',
  'availability': 'Year-round',
  'avg_score': '3.43',
  'beer_name': "Macardle's Ale",
  'brew_co': 'Macardle Moore',
  'description': 'No notes at this time.',
  'score': '81',
  'score_cat': 'good',
  'style': 'Irish Red Ale'},
 '/beer/profile/664/4025/': {'abv': '4.80%',
  'availability': 'Year-round',
  'avg_score': '3.48',
  'beer_name': 'Courage Directors Bitter',
  'brew_co': "Wells & Young's Ltd",
  'description': 'No notes at this time.',
  'score': '80',
  'score_cat': 'good',
  'style': 'English Bitter'},
 '/beer/profile/29/43929/': {'abv': '5.50%',
  'availability': 'Summer',
  'avg_score': '3.5',
  'beer_name': 'Michelob Dunkel Weisse',
  'brew_co': 'Anheuser-Busch',
  'description': 'No notes at this time.',
  'score': '80',
  'score_cat': 'good',
  'style': 'Dunkelweizen'},
 '/beer/profile/14064/35606/': {'abv': '4.50%',
  'availability': 'Year-round',
  'avg_score': '3.78',
  'beer_name': 'Isabella Blonde',
  'brew_co': '

In [18]:
f = open('beer_advocate_unique_beers.json','r')
j = json.loads(f.read())

In [19]:
#j = json.dumps(unique_beer_dict)
unique_beer_dict = j

In [589]:
f = open('beer_advocate_unique_beers.json','w')

In [590]:
f.write(j)

2851955

In [12]:
f.close()

In [27]:
client = MongoClient()

client = MongoClient()
db = client.beer_advocate_database

beer_col = db.unique_beers
rev_col = db.reviews

In [28]:
style_dict = {}
user_dict = defaultdict(list)


In [ ]:
for k in unique_beer_dict:
    sleep(.1)
    url = 'https://www.beeradvocate.com' + k
    resp = requests.get(url)

    soup = BeautifulSoup(resp.text,'lxml')
    soup = soup.find('div',{'class': 'mainContent'})

    beer_dict = {}
    beer = unique_beer_dict[k]['beer_name']
    beer_dict['beer_name'] = beer 
    beer_dict['brew_co'] = soup.find('div',{'class':'titleBar'}).text.split('|')[1].strip()
    beer_dict['score'] = soup.find('span',{'BAscore_big ba-score'}).text
    beer_dict['score_cat'] = soup.find('span',{'ba-score_text'}).text
    beer_dict['avg_score'] = soup.find('span',{'class': 'ba-ravg'}).text
    beer_dict['style'] = soup.find(text='Style:').parent.findNext('a').text
    
    style_num = soup.find(text='Style:').parent.findNext('a')['href'].split('style/')[1].strip('/')
    style_dict[style_num] = beer_dict['style']
    
    beer_dict['abv'] = soup.find(text=re.compile("ABV")).parent.nextSibling.strip()
    beer_dict['availability'] = soup.find(text=re.compile("Availability")).parent.nextSibling.strip()
    beer_dict['description'] = soup.find(text=re.compile("Commercial Description")).parent.findNext('br').nextSibling.strip()

    beer_col.insert({'url_id': k, 'content': beer_dict})
    
    ratings = soup.find('div', {'id': 'rating_fullview'})
    next_exists = True

    while next_exists:
        for comment in ratings.findAll('div', {'class': 'user-comment'}):
            review_dict = {}
            review_dict['usrid'] = comment['ba-user']
            review_dict['beer_url'] = k
            review_dict['score'] = comment.find('span', {'class': 'BAscore_norm'}).text
            if len(comment.findAll('span', {'class': 'muted'})) > 2:
                lstfo_score = [v.strip() for v in comment.find('span', {'class': 'muted'}).text.split('|')]
            else: 
                lstfo_score = 'none'
            review_dict['lstfo_score'] = lstfo_score
            review = comment.text.split('★')[0]
            review = ''.join(review.split('%')[1:])
            r = review.split('|')[-1]
            review = re.split('overall: \d\.?\d?\d?', r)[-1]
            review_dict['review'] = review
            date = comment.findAll('span', {'class':'muted'})[-1].findAll('a')[1].text
            review_dict['date'] = date
            
            rev_col.insert(review_dict)
            
        try:    
            if ratings.parent.find_all_previous('div')[1](text=re.compile('next')).parent.name == 'span':
                next_exists = False
            else:
                url = 'https://www.beeradvocate.com' + soup.find(text=re.compile('next')).parent['href']
        except:
            next_exists = False

        resp = requests.get(url)
        soup = BeautifulSoup(resp.text,'lxml')
        soup = soup.find('div',{'class': 'mainContent'})
        ratings = soup.find('div', {'id': 'rating_fullview'})
        sleep(.1)
        
    i += 1

/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:25: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/home/ubuntu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:49: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [605]:
 for k in user_dict:
    for i in range(len(user_dict[k])):
        if type(user_dict[k][i][-1]) == dt.datetime:
            user_dict[k][i][-1] = user_dict[k][i][-1].strftime('%b %d, %Y')

In [ ]:
j = json.dumps(user_dict)

In [1]:
f = open('beer_advocate_users.json','w')
f.write(j)

NameError: name 'json' is not defined

In [ ]:
f.close()
j = json.dumps(style_dict)
f = open('beer_advocate_style_dict.json','w')
f.write(j)

In [ ]:
f.close()

In [30]:
print('h')

h
